<a href="https://colab.research.google.com/github/andrewjh9/CenBench/blob/main/CenBench.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
#CenSET


In [48]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [49]:
# Author: Decebal Constantin Mocanu et al.;
# Proof of concept implementation of Sparse Evolutionary Training (SET) of Multi Layer Perceptron (MLP) on CIFAR10 using Keras and a mask over weights.
# This implementation can be used to test SET in varying conditions, using the Keras framework versatility, e.g. various optimizers, activation layers, tensorflow
# Also it can be easily adapted for Convolutional Neural Networks or other models which have dense layers
# However, due the fact that the weights are stored in the standard Keras format (dense matrices), this implementation can not scale properly.
# If you would like to build and SET-MLP with over 100000 neurons, please use the pure Python implementation from the folder "SET-MLP-Sparse-Python-Data-Structures"

# This is a pre-alpha free software and was tested with Python 3.5.2, Keras 2.1.3, Keras_Contrib 0.0.2, Tensorflow 1.5.0, Numpy 1.14;
# The code is distributed in the hope that it may be useful, but WITHOUT ANY WARRANTIES; The use of this software is entirely at the user's own risk;
# For an easy understanding of the code functionality please read the following articles.

# If you use parts of this code please cite the following articles:
#@article{Mocanu2018SET,
#  author =        {Mocanu, Decebal Constantin and Mocanu, Elena and Stone, Peter and Nguyen, Phuong H. and Gibescu, Madeleine and Liotta, Antonio},
#  journal =       {Nature Communications},
#  title =         {Scalable Training of Artificial Neural Networks with Adaptive Sparse Connectivity inspired by Network Science},
#  year =          {2018},
#  doi =           {10.1038/s41467-018-04316-3}
#}

#@Article{Mocanu2016XBM,
#author="Mocanu, Decebal Constantin and Mocanu, Elena and Nguyen, Phuong H. and Gibescu, Madeleine and Liotta, Antonio",
#title="A topological insight into restricted Boltzmann machines",
#journal="Machine Learning",
#year="2016",
#volume="104",
#number="2",
#pages="243--270",
#doi="10.1007/s10994-016-5570-z",
#url="https://doi.org/10.1007/s10994-016-5570-z"
#}

#@phdthesis{Mocanu2017PhDthesis,
#title = "Network computations in artificial intelligence",
#author = "D.C. Mocanu",
#year = "2017",
#isbn = "978-90-386-4305-2",
#publisher = "Eindhoven University of Technology",
#}\\\

# Alterations made by Andrew Heath


# Install requirements
# !pip3 uninstall tensorflow y
# !pip3 install tensorflow --user
# # !pip3 install --upgrade tensorflow
# !pip3 install graphviz
# !pip3 install pydot
# !pip3 install keras-visualizer
# !pip3 install cmake
# !pip3 install cython
!pip3 install networkit
!pip3 install networkx

# CenBench 
The a benchmark framework used to perform the expeirment

## Set up

In [50]:
from __future__ import division
from __future__ import print_function
import tensorflow as tf
from datetime import datetime
import time
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras import optimizers
from tensorflow.python.client import device_lib

import numpy as np
from numpy import asarray
from numpy import savetxt
import pydot
from tensorflow.keras import models, layers  
from tensorflow.keras import backend as K
from tensorflow.keras import layers
from tensorflow.keras import activations
from tensorflow.keras import utils as k_utils
import time
from copy import copy, deepcopy
import networkx.algorithms.isomorphism as iso
from  more_itertools import take
from scipy.sparse import dok_matrix
import networkx as nx
import networkit as nk
from random import sample


#Please note that in newer versions of keras_contrib you may encounter some import errors. You can find a fix for it on the Internet, or as an alternative you can try other activations functions.
# import tf.keras.activations.relu as SReLU
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.datasets import fashion_mnist 
from tensorflow.keras.utils import to_categorical
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
%matplotlib inline  

class Constraint(object):

    def __call__(self, w):
        return w

    def get_config(self):
        return {}

class MaskWeights(Constraint):

    def __init__(self, mask):
        self.mask = mask
        self.mask = K.cast(self.mask, K.floatx())

    def __call__(self, w):
        w = w.assign(w * self.mask)
        return w

    def get_config(self):
        return {'mask': self.mask}


def find_first_pos(array, value):
    idx = (np.abs(array - value)).argmin()
    return idx


def find_last_pos(array, value):
    idx = (np.abs(array - value))[::-1].argmin()
    return array.shape[0] - idx


def createWeightsMask(epsilon,noRows, noCols):
    # generate an Erdos Renyi sparse weights mask
    mask_weights = np.random.rand(noRows, noCols)
    prob = 1 - (epsilon * (noRows + noCols)) / (noRows * noCols)  # normal tp have 8x connections
    mask_weights[mask_weights < prob] = 0
    mask_weights[mask_weights >= prob] = 1
    print("Init mask weight shape: ",mask_weights.shape)
    noParameters = np.sum(mask_weights)
    print ("Create Sparse Matrix: No parameters, NoRows, NoCols ",noParameters,noRows,noCols)
    return [noParameters,mask_weights]

## Init & Parameters

In [51]:
class CenBench_MLP():
    def __init__(self, maxepoches, dataset, pruning_approach, batch_size = 100, centrality_metric=None, zeta=0.05):

        # Move
        def prod(val) : 
            res = 1 
            for ele in val: 
                res *= ele 
            return res 

        # Fetch the parameters for a given dataset
        dataset_name = dataset. __name__.split(".")[3]

        self.hidden_layer_sizes, self.num_classes, self.dataset_input_shape = get_dataset_params(dataset_name)



        # set model parameters
        self.number_of_connections_per_epoch = 0
        self.layer_sizes = [prod(self.dataset_input_shape), self.hidden_layer_sizes[0], self.hidden_layer_sizes[1], self.hidden_layer_sizes[2]]
        self.epsilon = 20 # control the sparsity level as discussed in the paper
        self.zeta = zeta # the fraction of the weights removed
        self.batch_size = batch_size # batch sgenerate_weights_matrix_from_networkize
        self.maxepoches = maxepoches     # number of epochs
        self.learning_rate = 0.01 # SGD learning rate
        self.momentum = 0.9 # SGD momentum
        self.dataset = dataset
        self.pruning_approach = pruning_approach
        self.centrality_metric = centrality_metric
        self.current_accuracy = 0 #Only for AccSET
        self.k = -0.27 #Only for AccSET
        self.number_of_connections_per_epoch = 0#Only for AccSET
        self.current_epoc = 0
        self.mean_kc_scores = []
        self.mean_l_scores =[]
        [self.noPar1, self.wm1] = createWeightsMask(self.epsilon, prod(self.dataset_input_shape), self.hidden_layer_sizes[0])
        [self.noPar2, self.wm2] = createWeightsMask(self.epsilon,self.hidden_layer_sizes[0], self.hidden_layer_sizes[1])
        [self.noPar3, self.wm3] = createWeightsMask(self.epsilon,self.hidden_layer_sizes[1], self.hidden_layer_sizes[2])
        print("Total noPars: ", self.noPar1 + self.noPar2 + self.noPar3)
        # initialize layers weightsnk
        self.w1 = None
        self.w2 = None
        self.w3 = None
        self.w4 = None

        # initialize weights for SReLu activation function
        self.wSRelu1 = None
        self.wSRelu2 = None
        self.wSRelu3 = None

        # create a SET-MLP model
        self.create_model()


In [52]:
def get_dataset_params(dataset_name):

    if dataset_name == "cifar10":
        hidden_layer_sizes = [4000,1000,4000]
        num_classes = 10
        dataset_input_shape = (32, 32, 3)
        return hidden_layer_sizes, num_classes, dataset_input_shape

    elif dataset_name == "cifar100":
        hidden_layer_sizes = [4000,1000,4000]
        num_classes = 100
        dataset_input_shape = (32, 32, 3)
        return hidden_layer_sizes, num_classes, dataset_input_shape

    elif dataset_name == "fashion_mnist":
        hidden_layer_sizes = [256, 128, 100]
        num_classes = 10
        dataset_input_shape = (28,28,1) 
        return hidden_layer_sizes, num_classes, dataset_input_shape

    elif dataset_name == "higgs":
        hidden_layer_sizes, num_classes, dataset_input_shape = None, None, None
        print("Dataset HIGGS not implemented !")
        return hidden_layer_sizes, num_classes, dataset_input_shape



## Create model

In [53]:
class CenBench_MLP(CenBench_MLP):
    def create_model(self):

        # create a SET-MLP model for CIFAR10 with 3 hidden layers
        self.model = Sequential()
        #Input layer ---  
        self.model.add(Flatten(input_shape=self.dataset_input_shape))
        
        # Hidden layer 1
        self.model.add(Dense(self.hidden_layer_sizes[0], name="sparse_1",kernel_constraint=MaskWeights(self.wm1),weights=self.w1))
        self.model.add(layers.Activation(activations.relu,name="srelu1",weights=self.wSRelu1))
        self.model.add(Dropout(0.3))#Helps with overfitting, only present in training
        # Hidden layer 2
        self.model.add(Dense(self.hidden_layer_sizes[1], name="sparse_2",kernel_constraint=MaskWeights(self.wm2),weights=self.w2))
        self.model.add(layers.Activation(activations.relu,name="srelu2",weights=self.wSRelu2))
        self.model.add(Dropout(0.3))#Helps with overfitting, only present in training
        # Hidden layer 3
        self.model.add(Dense(self.hidden_layer_sizes[2], name="sparse_3",kernel_constraint=MaskWeights(self.wm3),weights=self.w3))
        self.model.add(layers.Activation(activations.relu,name="srelu3",weights=self.wSRelu3))
        self.model.add(Dropout(0.3)) #Helps with overfitting, only present in training
        # Output layer
        self.model.add(Dense(self.num_classes, name="dense_4",weights=self.w4)) #please note that there is no need for a sparse output layer as the number of classes is much smaller than the number of input hidden neurons
        self.model.add(Activation('softmax'))

## Rewrite Weight Mask SET

In [54]:
class CenBench_MLP(CenBench_MLP):
    def rewireMask_SET(self, weights, noWeights):

        # rewire weight matrix
        # remove zeta largest negative and smallest positive weights
        values = np.sort(weights.ravel())
        firstZeroPos = find_first_pos(values, 0)
        lastZeroPos = find_last_pos(values, 0)
        largestNegative = values[int((1-self.zeta) * firstZeroPos)]
        smallestPositive = values[int(min(values.shape[0] - 1, lastZeroPos +self.zeta * (values.shape[0] - lastZeroPos)))]
        rewiredWeights = weights.copy();
        rewiredWeights[rewiredWeights > smallestPositive] = 1;
        rewiredWeights[rewiredWeights < largestNegative] = 1;
        rewiredWeights[rewiredWeights != 1] = 0;
        weightMaskCore = rewiredWeights.copy()  

        # add zeta random weights

        nrAdd = 0
        sum_layer = np.sum(rewiredWeights)
        noRewires = noWeights - sum_layer
        while (nrAdd < noRewires):
            i = np.random.randint(0, rewiredWeights.shape[0])
            j = np.random.randint(0, rewiredWeights.shape[1])
            if (rewiredWeights[i, j] == 0):
                rewiredWeights[i, j] = 1
                nrAdd += 1
        print("Num removed (and then added) ", nrAdd)
        self.number_of_connections_per_epoch = self.number_of_connections_per_epoch + np.sum(rewiredWeights)
        return [rewiredWeights, weightMaskCore]


## Rewrite weight mask CenSET

In [55]:
class CenBench_MLP(CenBench_MLP):
    def rewireMask_CenSET(self, layer_weights, layer_weights_mask, noWeights):

        nkG = generate_NN_network(self.layer_sizes, layer_weights, layer_weights_mask)  
    
        predicted_mean_lap  = 0.02 * self.current_epoch
        nodes_to_remove = self.find_nodes_lowest_centraility(nkG, self.zeta * 100, self.centrality_metric)
        print("num nodes removed: ", len(nodes_to_remove))


        rewiredWeights = generate_weight_masks_matrix_from_network(self.layer_sizes, nkG, layer_weights_mask, nodes_to_remove)
        # rewiredWeights = [[[step(x) for x in inner] for inner in layer]  for layer in rewiredWeights]
        weightMaskCore = deepcopy(rewiredWeights)
        # print("Shape rewieredWeights: ", rewiredWeights)
        for layer_i, layer in enumerate(rewiredWeights):
            nrAdd = 0
            layer_sum = layer.sum()
            noRewires = noWeights[layer_i] - layer_sum
            while (nrAdd < noRewires):
                i = np.random.randint(0, len(layer))
                j = np.random.randint(0, len(layer[0]))
                if (layer[i][j] == 0):
                    layer[i][j] = 1
                    nrAdd += 1
        self.number_of_connections_per_epoch = self.number_of_connections_per_epoch + np.sum(rewiredWeights[0]) + np.sum(rewiredWeights[1]) + np.sum(rewiredWeights[2])
        return [rewiredWeights, weightMaskCore]

In [56]:
class CenBench_MLP(CenBench_MLP):
    def rewireMask_AccSET(self, weights, no_weights):
                # rewire weight matrix
        # remove zeta largest negative and smallest positive weights
        values = np.sort(weights.ravel())
        first_zero_pos = find_first_pos(values, 0)
        last_zero_pos = find_last_pos(values, 0)
        largest_negative = values[int((1 - self.zeta) * first_zero_pos)]
        smallest_positive = values[
            int(min(values.shape[0] - 1, last_zero_pos + self.zeta * (values.shape[0] - last_zero_pos)))]
        rewired_weights = weights.copy()
        rewired_weights[rewired_weights > smallest_positive] = 1
        rewired_weights[rewired_weights < largest_negative] = 1
        rewired_weights[rewired_weights != 1] = 0
        weight_mask_core = rewired_weights.copy()

        # add zeta random weights
        nr_add = 0
        # Number of connections to be added
        no_rewires = self.calculate_number_of_connections_to_add(rewired_weights, no_weights)
        while nr_add < no_rewires:
            i = np.random.randint(0, rewired_weights.shape[0])
            j = np.random.randint(0, rewired_weights.shape[1])
            if rewired_weights[i, j] == 0:
                rewired_weights[i, j] = 1
                nr_add += 1
        self.number_of_connections_per_epoch = self.number_of_connections_per_epoch + np.sum(rewired_weights)

        return [rewired_weights, weight_mask_core]

In [57]:
 class CenBench_MLP(CenBench_MLP):
    def calculate_number_of_connections_to_add(self, rewired_weights, no_weights):
        # Adjust sparsity based on the accuracy
        x = self.current_accuracy
        y = 1 - ((x - x * self.k) / (self.k - abs(x) * 2 * self.k + 1))
        return (no_weights - np.sum(rewired_weights)) * y

## Testing conversion
A method for testing if the conversion between weights and graphs is working as intended

In [58]:
def testing_conversion(layer_weights, layer_weight_mask, iterations):
    graphs = []
    layer_weight_masks = []
    # For debug remove
    count_of_all_weights = 0
    for layer in layer_weights:
        count_of_all_weights += np.count_nonzero(layer)
    print("Inital number of weights in arrays: ", count_of_all_weights)
    for i in range(0, iterations):
        nkG = generate_NN_network(self.layer_sizes, layer_weights, layer_weight_mask)  
        layer_weight_mask = generate_weight_masks_matrix_from_network(self.layer_sizes, nkG, layer_weight_mask, [])
        graphs.append(nkG)
        layer_weight_masks.append(layer_weight_mask)
    print("finished with building set")
    i = 0
    for first, second in zip(graphs, graphs[1:]):
        print("Comparison: ", i)
        num_edges_1 = nk.nxadapter.nk2nx(first).number_of_edges()
        num_edges_2 = nk.nxadapter.nk2nx(second).number_of_edges()
        print("Num edges: ",num_edges_1,  num_edges_2)
        sum_edges_1 = layer_weight_masks[i][0].sum()+ layer_weight_masks[i][1].sum()+ layer_weight_masks[i][2].sum()
        sum_edges_2 = layer_weight_masks[i+1][0].sum()+ layer_weight_masks[i+1][1].sum()+ layer_weight_masks[i+1][2].sum()
        print("Num edges: ",num_edges_1,  num_edges_2)
        print("Inital number of non zero weights: ", count_of_all_weights)
        print("Sum edges weights",sum_edges_1, sum_edges_2 )
        comparison = layer_weight_masks[i][0] == layer_weight_masks[i+1][0]
        equal_layers = comparison.all()
        if not equal_layers:
            print("layers are not the same")
            print(layer_weight[i][0][100:200])
            print(layer_weight[i+1][0][100:200])
            return False

        em = iso.categorical_edge_match('weight', 'weight')
        if not (nx.algorithms.isomorphism.is_isomorphic(nk.nxadapter.nk2nx(first),nk.nxadapter.nk2nx(second), edge_match=em)):     
            print("ISO failed")
            return False
        elif not (num_edges_1 ==num_edges_2 ):     
            print("Edge number changes")
            return False
        i = i + 1
    return True


## Find Centrailities 

In [59]:
class CenBench_MLP(CenBench_MLP):
    def visualise(self):
        visualizer(self.model, view=True)


## Weight evolution

In [60]:
class CenBench_MLP(CenBench_MLP):
    def weightsEvolution(self):
        # this represents the core of the CenSET procedure. It removes the weights closest to zero in each layer and add new random weights
        # The mask shows which nodes have been removed 
        # The weights shows the  
        self.w1 = self.model.get_layer("sparse_1").get_weights()
        self.w2 = self.model.get_layer("sparse_2").get_weights()
        self.w3 = self.model.get_layer("sparse_3").get_weights()
        self.w4 = self.model.get_layer("dense_4").get_weights()
        # testing_conversion([self.w1[0], self.w2[0], self.w3[0]], [self.wm1, self.wm2, self.wm3], 10)
        # return

        self.wSRelu1 = self.model.get_layer("srelu1").get_weights()
        self.wSRelu2 = self.model.get_layer("srelu2").get_weights()
        self.wSRelu3 = self.model.get_layer("srelu3").get_weights()

        if(self.pruning_approach == "SET"):
            print("------------------------SET -------------------")
            [self.wm1, self.wm1Core] = self.rewireMask_SET(self.w1[0], self.noPar1)
            [self.wm2, self.wm2Core] = self.rewireMask_SET(self.w2[0], self.noPar2)
            [self.wm3, self.wm3Core] = self.rewireMask_SET(self.w3[0], self.noPar3)


            G = generate_NN_network(self.hidden_layer_sizes, [self.w1[0], self.w2[0], self.w3[0]],[self.wm1, self.wm2, self.wm3])  
            btwn = nk.centrality.LaplacianCentrality(G, normalized=False)
            btwn.run()
            scores_cen = [i[1] for i in btwn.ranking()]
            # savetxt("results/node_centrality_scores_set/"+str(self.current_epoch)+"laplacian.csv", asarray(scores_cen), delimiter=',')
            self.mean_l_scores.append(np.mean(scores_cen))
      

        elif(self.pruning_approach == "AccSET"):
            print("------------------------AccSET -------------------")
            [self.wm1, self.wm1Core] = self.rewireMask_AccSET(self.w1[0], self.noPar1)
            [self.wm2, self.wm2Core] = self.rewireMask_AccSET(self.w2[0], self.noPar2)
            [self.wm3, self.wm3Core] = self.rewireMask_AccSET(self.w3[0], self.noPar3)

        elif("CenSET" == self.pruning_approach):    
            print("------------------------CenSET -------------------")

            [self.wm1, self.wm2, self.wm3], [self.wm1Core, self.wm2Core, self.wm3Core] = self.rewireMask_CenSET([self.w1[0], self.w2[0], self.w3[0]],[self.wm1, self.wm2, self.wm3] ,[self.noPar1, self.noPar2, self.noPar3])

        else:
            raise ValueError("Unsupported pruning approach:"+self.pruning_approach)
        
        self.w1[0] = self.w1[0] * self.wm1Core
        self.w2[0] = self.w2[0] * self.wm2Core
        self.w3[0] = self.w3[0] * self.wm3Core
       
 
 



## Read dataset

In [61]:
class CenBench_MLP(CenBench_MLP):
    def read_data(self):
        # May need rewriting
        (x_train, y_train), (x_test, y_test) = self.dataset.load_data()
        y_train = to_categorical(y_train, self.num_classes)
        y_test = to_categorical(y_test, self.num_classes)
        x_train = x_train.astype('float32')
        x_test = x_test.astype('float32')
        # reshape dataset to have a single channel fashionmist
        print("Dataset name: ", self.dataset.__name__.split(".")[3])
        if self.dataset.__name__.split(".")[3] == "fashion_mnist":
            x_train = x_train.reshape((x_train.shape[0], 28, 28, 1))
            x_test = x_test.reshape((x_test.shape[0], 28, 28, 1))  
        #normalize data
        xTrainMean = np.mean(x_train, axis=0)
        xTtrainStd = np.std(x_train, axis=0)
        x_train = (x_train - xTrainMean) / xTtrainStd
        x_test = (x_test - xTrainMean) / xTtrainStd

        return [x_train, x_test, y_train, y_test]

## Training


In [62]:
class CenBench_MLP(CenBench_MLP):
    def train(self):
        # read CIFAR10 data
        [x_train,x_test,y_train,y_test]=self.read_data()
        #data augmentation
        datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
            width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=True,  # randomly flip images
            vertical_flip=False)  # randomly flip images
        datagen.fit(x_train)

        self.model.summary()

        # training process in a for loop
        self.accuracies_per_epoch=[]
        self.loss_per_epoch=[]
        self.connections_per_epoch=[]
        for epoch in range(0, self.maxepoches):
            self.current_epoch = epoch
            self.number_of_connections_per_epoch = 0.0
            print("Enter epoch: ", epoch)
            sgd = optimizers.SGD(lr=self.learning_rate, momentum=self.momentum)
            self.model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

            history = self.model.fit(datagen.flow(x_train, y_train,
                                                batch_size=self.batch_size),
                            steps_per_epoch=x_train.shape[0]//self.batch_size,
                                epochs=epoch,
                                validation_data=(x_test, y_test),
                                    initial_epoch=epoch-1)
            # print(history.history.)
            self.current_accuracy = history.history['val_accuracy'][0]

        #    self.loss_per_epoch.append(history.history['val_loss'])
        #    self.connections_per_epoch.append()
            #ugly hack to avoid tensorflow memory increase for multiple fit_generator calls. Theano shall work more nicely this but it is outdated in general

            # Tracking current accuracy for AccSET and possible exentions
            self.accuracies_per_epoch.append(history.history['val_accuracy'][0])
            self.loss_per_epoch.append(history.history["val_loss"])
            self.weightsEvolution()
            print("adding to connections per epoch: ", self.number_of_connections_per_epoch)
            self.connections_per_epoch.append(self.number_of_connections_per_epoch)
            K.clear_session()
            self.create_model()

        # savetxt("/content/drive/My Drive/bsc_thesis_colab_results/results/node_centrality_scores_set/"+"KatzCentrality_"+time.strftime("%Y%m%d-%H%M%S")+".csv", asarray(self.mean_kc_scores), delimiter=',')


        return [self.accuracies_per_epoch,  self.connections_per_epoch, self.loss_per_epoch, self.mean_l_scores]
           


## Generate Network from From weight array

In [63]:
# TODO change this to only use networkit
# TODO change to use a lil sparse representation as this will likely be faster
def generate_NN_network(layers, layer_weights, layer_weights_mask):
    iterations = 0
    n_nodes = sum(layers)
    adj_matrix = dok_matrix((n_nodes, n_nodes), dtype=np.float32)
    start = time.time()
    for layer_i, layer in enumerate(layers):    
        if not layer_i == len(layers) - 1 :
            # Multiply the current layer by the weight mask to remove nodes, TODO check this
            sparse_layer_weights = layer_weights[layer_i] * layer_weights_mask[layer_i]
          

            current_layer_start_offset = 0 if layer_i == 0 else sum(layers[0 : layer_i])
            current_layer_end_offset = current_layer_start_offset + layer - 1
            next_layer_start_offset = current_layer_end_offset + 1 
            next_layer_end_offset = next_layer_start_offset +  layers[layer_i + 1] -1

            layer_index_value_dic = {(x + current_layer_start_offset, y + next_layer_start_offset):value for (x ,y), value in np.ndenumerate(sparse_layer_weights) if not value == 0 }

            adj_matrix._update(layer_index_value_dic)

    print("W -> N  time: s",(time.time() - start))
    
    G = nx.convert_matrix.from_scipy_sparse_matrix(adj_matrix, create_using=nx.DiGraph, edge_attribute='weight')
    Gnk = nk.nxadapter.nx2nk(G, weightAttr="weight")
    return  Gnk


## Generate Weight Arrays from Network

In [64]:
def generate_weight_masks_matrix_from_network(layers, network, layer_weights_mask, nodes_to_remove):
    n_nodes = sum(layers)
    start = time.time()

    print("Layer weight mask before removal: ", np.count_nonzero(layer_weights_mask[0]) + np.count_nonzero(layer_weights_mask[1]) + np.count_nonzero(layer_weights_mask[2]))
    start = time.time()
    for layer_i, layer in enumerate(layers):    
        if layer_i == len(layers) - 1 :
            break

        current_layer_start_offset = 0 if layer_i == 0 else sum(layers[0 : layer_i])
        current_layer_end_offset = current_layer_start_offset + layer - 1
        next_layer_start_offset = current_layer_end_offset + 1 
        next_layer_end_offset = next_layer_start_offset +  layers[layer_i + 1] -1
       
        nodes_to_remove_in_layer_row  = [(remove  - current_layer_start_offset)  for remove in nodes_to_remove if remove >= current_layer_start_offset and remove <= current_layer_end_offset]
        nodes_to_remove_in_layer_col  = [(remove  - next_layer_start_offset)  for remove in nodes_to_remove if remove >= next_layer_start_offset and remove <= next_layer_end_offset]


        layer_weights_mask[layer_i][: , nodes_to_remove_in_layer_col] = 0
        layer_weights_mask[layer_i][nodes_to_remove_in_layer_row, :] = 0


    print("Layer weight mask after removal: ", np.count_nonzero(layer_weights_mask[0]) + np.count_nonzero(layer_weights_mask[1]) + np.count_nonzero(layer_weights_mask[2]))
    print("N -> W: ",(time.time() - start))
    return layer_weights_mask

# Find nodes with lowest centraility

In [65]:
class CenBench_MLP(CenBench_MLP):
  def find_nodes_lowest_centraility(self, G, removal_percentile, centrality_metric):
      if centrality_metric == "laplacian":
          if removal_percentile == 0.0:
            return []
          btwn = nk.centrality.LaplacianCentrality(G, normalized=False)
          btwn.run()
          ranking = btwn.ranking() 
          scores_cen = [i[1] for i in ranking ]
          mean = np.mean(scores_cen, axis=0)
          sd_scores_cen = np.std(scores_cen, axis=0)
      
          removed_via_sd = [node_num  for (node_num, cen_score) in ranking if (cen_score < mean -  1.5 * sd_scores_cen)]

          return removed_via_sd
           
          # removal_percentile_val = np.percentile(scores_cen, removal_percentile)
          # print("percentile value:", removal_percentile_val)
          #     # savetxt("results/node_centrality_scores_set/"+str(self.current_epoch)+"laplacian.csv", asarray(scores_cen), delimiter=',')
          # self.mean_l_scores.append(np.mean(scores_cen))

          # return [node_num  for (node_num, cen_score) in ranking if cen_score < removal_percentile_val]


      else:
        print("------------------------FAILED NO METRIC PASSED-----------------------------")
      # elif centrality_metric == "katz":
      #     btwn = nk.centrality.KatzCentrality(G)
      #     btwn.run()
      #     return btwn.ranking()[-number_of_nodes:]
      # elif centrality_metric == "pagerank":
      #     btwn = nk.centrality.PageRank(G)
      #     btwn.run()
      #     return btwn.ranking()[-number_of_nodes:]
      # elif centrality_metric == "topharmonic":
      #     btwn = nk.centrality.TopHarmonicCloseness(G)
      #     btwn.run()
      #     return btwn.ranking()[-number_of_nodes:]

  def getDateTime():
      now = datetime.now()
      timestamp = datetime.timestamp(now)
      return datetime.fromtimestamp(timestamp)  
     

# Plot accuracy

In [66]:
def plot_save_accuracy(title, results_accu, results_connections, results_loss, results_cen , dataset_name, pruning_approach, epochs, centrality_metric=None, zeta = None, tag=None):
    if centrality_metric is not None:
        save_name = pruning_approach +"_"+centrality_metric+"_"+dataset_name+"_for_"+str(epochs)+"_epochs_"+time.strftime("%Y%m%d-%H%M%S")
    else:
         save_name = pruning_approach +"__"+dataset_name+"_for_"+str(epochs)+"_epochs_"+time.strftime("%Y%m%d-%H%M%S")
    if zeta is not None:
         save_name = save_name + "_zeta_" + str(zeta)
    tag = str(tag) if tag else ""
    savetxt("/content/drive/My Drive/bsc_thesis_colab_results/results/"+save_name+"_accuracy_"+tag+".csv", asarray(results_accu), delimiter=',')
    savetxt("/content/drive/My Drive/bsc_thesis_colab_results/results/"+save_name+"_connections"+tag+".csv", asarray(results_connections), delimiter=',')
    savetxt("/content/drive/My Drive/bsc_thesis_colab_results/results/"+save_name+"_loss"+tag+".csv", asarray(results_loss), delimiter=',')
    savetxt("/content/drive/My Drive/bsc_thesis_colab_results/results/"+save_name+"_lap"+tag+".csv", asarray(results_cen), delimiter=',')


# Run experiments
A method for running multiple experiments

In [67]:
def run_experiments(datasets, maxepoches, pruning_approachs, experiment_titles, centrality_metrics=None, tags=None):
    if  len(datasets) == len(maxepoches) == len(pruning_approachs) == len(experiment_titles)  :
        for experiment_i, experiment_title in enumerate(experiment_titles):
            dataset_name = datasets[experiment_i]. __name__.split(".")[3]
            print("------------START of experiment '"+experiment_title+"' for dataset: "+dataset_name+"------------")
            smlp = CenBench_MLP(maxepoches=maxepoches[experiment_i], dataset=datasets[experiment_i], pruning_approach=pruning_approachs[experiment_i],centrality_metric=centrality_metrics[experiment_i] )
            # Saving results
            [res_acc, res_conn, res_loss, res_cen] = smlp.train()
            plot_save_accuracy(experiment_title, res_acc, res_conn, res_loss,res_cen, dataset_name,pruning_approachs[experiment_i], maxepoches[experiment_i], centrality_metrics[experiment_i], tags[experiment_i] )
            
            print("------------END of experiment '"+experiment_title+"' for dataset: "+dataset_name+"------------")
    else:
        raise ValueError("Incorrect experiment setup")

## Fit Zeta

In [68]:
def fit_zeta(maxepoches, dataset, pruning_approach, experiment_title, zeta_range, zeta_step, centrality_metric=None, tag= None):
    for zeta in np.arange(zeta_range[0], zeta_range[1], zeta_step):
        dataset_name = dataset. __name__.split(".")[3]
        smlp = CenBench_MLP(maxepoches=maxepoches, dataset=dataset, pruning_approach=pruning_approach, centrality_metric=centrality_metric, zeta= zeta)
        # Saving results
        [res_acc, res_conn, res_loss, res_cen] = smlp.train()
        plot_save_accuracy(experiment_title,res_acc, res_conn, res_loss,res_cen, dataset_name ,pruning_approach, maxepoches, centrality_metric, str(zeta), tag )

# Configure Experiments - Start Experiments
Configure the Experiments and run them

In [ ]:
K.clear_session()

print(device_lib.list_local_devices())
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

datasets=[fashion_mnist] 

maxepoches=[100]
pruning_approachs=["CenSET"]
centrality_metrics = ['laplacian',]
experiment_titles = ["Model accuracy vs Centrality using SET"]
tags = ["1_5_sds"]
run_experiments(datasets, maxepoches, pruning_approachs, experiment_titles, centrality_metrics, tags)

# fit_zeta(100, cifar10, "CenSET", "Model accuracy using SET", (0.03, 0.11), 0.01,"laplacian", "NEW_CenSET_Percentile" )


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9672628706212989174
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15395979264
locality {
  bus_id: 1
  links {
  }
}
incarnation: 14086484443075370737
physical_device_desc: "device: 0, name: Tesla V100-SXM2-16GB, pci bus id: 0000:00:04.0, compute capability: 7.0"
]
Num GPUs Available:  1
------------START of experiment 'Model accuracy vs Centrality using SET' for dataset: fashion_mnist------------
Init mask weight shape:  (784, 256)
Create Sparse Matrix: No parameters, NoRows, NoCols  21191.0 784 256
Init mask weight shape:  (256, 128)
Create Sparse Matrix: No parameters, NoRows, NoCols  7728.0 256 128
Init mask weight shape:  (128, 100)
Create Sparse Matrix: No parameters, NoRows, NoCols  4556.0 128 100
Total noPars:  33475.0
Dataset name:  fashion_mnist
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape    

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


600/600 [==============================] - 15s 24ms/step - loss: 1.3481 - accuracy: 0.5019 - val_loss: 0.7557 - val_accuracy: 0.7222
------------------------CenSET -------------------
W -> N  time: s 0.1674039363861084
num nodes removed:  24
Layer weight mask before removal:  33475
Layer weight mask after removal:  31444
N -> W:  0.001157522201538086
adding to connections per epoch:  33475.0
Enter epoch:  1
600/600 [==============================] - 15s 24ms/step - loss: 0.9142 - accuracy: 0.6634 - val_loss: 0.6713 - val_accuracy: 0.7491
------------------------CenSET -------------------
W -> N  time: s 0.16329503059387207
num nodes removed:  26
Layer weight mask before removal:  33475
Layer weight mask after removal:  31475
N -> W:  0.001024007797241211
adding to connections per epoch:  33475.0
Enter epoch:  2
Epoch 2/2
600/600 [==============================] - 14s 23ms/step - loss: 0.8447 - accuracy: 0.6866 - val_loss: 0.6327 - val_accuracy: 0.7684
------------------------CenSET ---



### Tickets
- How to find the inverse function, find where the centraility stops increasing this is 100% of centraility, then the centraility measures can become a normalised percentage based on this. Then there is a centraility percentage and epoch function. This can be used to scale the pruning rate. The function should be reverse compare to the one seen in the data.
- The fuction of the rate of removal of nodes should be the inverse of the function of the increase of centraility observered
    - SET on FashionMNST should be rerun recording lap centraility
    - Perhaps 2 More datasets should be run recording lap centraility
    - Using all of these datasets I can try and come up with a matching function 
    - Possible candidates: https://en.wikipedia.org/wiki/Exponential_growth#/media/File:Exponential.svg x^3 looks good 

- Improve access speed on sparse adj matrix in W -> N - test using list of list sparse matrices
- Read into Lap centraility
    - Does it work for directed graphs ? 
    - 
 
- Allow for changing of metric
- At each epoch in SET record the ranking of centraility
- use above to determine a centraility threshold to prune beneth.
- Choose better metrics
- Create framework to find pruning threshold for a metric
- Fix tex saving
- Show MLP in comparison charts ?
- Track number of connections per epoch
- Track number of connections and centraility across network at end of training
- Convert between iterations on SET to check conversion methods
- Get VPN 
- Get collab Pro
- Set up Collab with github: https://towardsdatascience.com/google-drive-google-colab-github-dont-just-read-do-it-5554d5824228

### Broken
- FashionMNST is not supported




